In [73]:
import tqdm as notebook_tqdm
import ipywidgets as widgets
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
import evaluate
from torchtext.data.utils import get_tokenizer
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration
from tqdm import tqdm
import torch
import math
import torch.nn as nn
import torch.optim as optim
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from datasets import load_dataset
from torch.nn import Transformer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from timeit import default_timer as timer
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

In [12]:
import pandas as pd

In [2]:
train_words = load_dataset("wmt16", "de-en", split="train[:50000]")
eval_words = load_dataset("wmt16", "de-en", split="validation")
test_words = load_dataset("wmt16", "de-en", split="test")

Generating train split:   0%|          | 0/4548885 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2169 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2999 [00:00<?, ? examples/s]

# Creating temp csv


In [16]:
de_list=[]
for i in range(100):
    de_list.append(train_words[i]['translation']['de'])
df = pd.DataFrame(de_list, columns=['de'])
df.to_csv("sample_infer_de_input.csv",index=False)

In [17]:
en_list=[]
for i in range(100):
    en_list.append(train_words[i]['translation']['en'])
df = pd.DataFrame(en_list, columns=['en'])
df.to_csv("sample_infer_en_input.csv",index=False)

# Helper for Part-A

In [67]:
#Environment variables
SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

In [71]:
de_tokenizer = get_tokenizer("spacy", language="de_core_news_sm")
en_tokenizer = get_tokenizer("spacy", language="en_core_web_sm")

In [79]:
def yield_tokens(data_iter, language):

    for data_sample in data_iter:
        if (language == "en"):
            yield en_tokenizer(data_sample["translation"]["en"])
        else:
            yield en_tokenizer(data_sample["translation"]["de"])


de_vocab = build_vocab_from_iterator(yield_tokens(train_words, SRC_LANGUAGE), specials=["<unk>", "<pad>", "<bos>", "<eos>"])
en_vocab = build_vocab_from_iterator(yield_tokens(train_words, TGT_LANGUAGE), specials=["<unk>", "<pad>", "<bos>", "<eos>"])
de_vocab.set_default_index(de_vocab["<unk>"])
en_vocab.set_default_index(en_vocab["<unk>"])

In [80]:
token_transform = {}
vocab_transform = {}
token_transform[SRC_LANGUAGE] = de_tokenizer
token_transform[TGT_LANGUAGE] = en_tokenizer
vocab_transform[SRC_LANGUAGE] = de_vocab
vocab_transform[TGT_LANGUAGE] = en_vocab
vocab_transform[SRC_LANGUAGE].set_default_index(vocab_transform[SRC_LANGUAGE]["<unk>"])
vocab_transform[TGT_LANGUAGE].set_default_index(vocab_transform[TGT_LANGUAGE]["<unk>"])

In [74]:
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: torch.Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

In [83]:
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: torch.Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

In [75]:
class Seq2SeqTransformer(nn.Module):
    def __init__(
        self,
        num_encoder_layers: int,
        num_decoder_layers: int,
        emb_size: int,
        nhead: int,
        src_vocab_size: int,
        tgt_vocab_size: int,
        dim_feedforward: int = 512,
        dropout: float = 0.1,
    ):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(
            d_model=emb_size,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
        )
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)

    def forward(
        self,
        src: torch.Tensor,
        trg: torch.Tensor,
        src_mask: torch.Tensor,
        tgt_mask: torch.Tensor,
        src_padding_mask: torch.Tensor,
        tgt_padding_mask: torch.Tensor,
        memory_key_padding_mask: torch.Tensor,
    ):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(
            src_emb,
            tgt_emb,
            src_mask,
            tgt_mask,
            None,
            src_padding_mask,
            tgt_padding_mask,
            memory_key_padding_mask,
        )
        return self.generator(outs)

    def encode(self, src: torch.Tensor, src_mask: torch.Tensor):
        return self.transformer.encoder(
            self.positional_encoding(self.src_tok_emb(src)), src_mask
        )

    def decode(self, tgt: torch.Tensor, memory: torch.Tensor, tgt_mask: torch.Tensor):
        return self.transformer.decoder(
            self.positional_encoding(self.tgt_tok_emb(tgt)), memory, tgt_mask
        )

In [76]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device = device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=device).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [81]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 32
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3


In [88]:
# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(device)
    src_mask = src_mask.to(device)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)
    for i in range(max_len-1):
        memory = memory.to(device)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(device)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [90]:
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

def tensor_transform(token_ids: list[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln],
                                               vocab_transform[ln],
                                               tensor_transform) 


def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))
    

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

# Infer For Part 2-A

In [92]:
transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)


C:\Users\Deeptanshu Barman\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [93]:
transformer.load_state_dict(torch.load("transformer.pth"))
transformer = transformer.to(device)

## sentence-wise

In [95]:
sentence="Hallo, ich bin hier"
print(translate(transformer, sentence))

 I am in favour of this . 


## File-wise

In [97]:
FILE_PATH="sample_infer_de_input.csv"
german_sentences=pd.read_csv("sample_infer_de_input.csv")['de'].tolist()
eng_sentences=[]
for i in tqdm(german_sentences):
    eng_sentences.append(translate(transformer, i))
df = pd.DataFrame({'en': eng_sentences,'de': german_sentences})
df.to_csv('output_parta.csv', index=False)

100%|██████████| 100/100 [00:53<00:00,  1.89it/s]


# Infer For Part 2-B

In [33]:
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [52]:
source_lang = "en"
target_lang = "de"
prefix = "translate English to German: "

def translate_to_german(english_sentence):
    input_ids = tokenizer.encode(prefix + english_sentence, return_tensors="pt")
    output_ids = model.generate(input_ids, max_length=100, num_beams=4, num_beam_groups=2, diversity_penalty=0.5, early_stopping=True, num_return_sequences=1, 
                              eos_token_id=tokenizer.eos_token_id, do_sample=False, top_k=50, top_p=0.95)
    german_translation = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return german_translation

## Sentence Wise

In [53]:
sentence="Resumption of the session"
translate_to_german(sentence)

'Wiederaufnahme der Sitzungsperiode'

## File

In [59]:
FILE_PATH="sample_infer_en_input.csv"
eng_sentences=pd.read_csv("sample_infer_en_input.csv")['en'].tolist()
german_sentences=[]
for i in tqdm(eng_sentences):
    german_sentences.append(translate_to_german(i))
df = pd.DataFrame({'en': eng_sentences,'de': german_sentences})
df.to_csv('output_partb.csv', index=False)

  0%|          | 0/100 [00:00<?, ?it/s]

C:\Users\Deeptanshu Barman\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\generation\configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
100%|██████████| 100/100 [05:24<00:00,  3.25s/it]


# Infer for Part 2-c

In [60]:
checkpoint = "checkpoint-7500"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [61]:
source_lang = "de"
target_lang = "en"
prefix = "translate German to English: "

def translate_to_english(german_sentence):
    input_ids = tokenizer.encode(prefix + german_sentence, return_tensors="pt")
    output_ids = model.generate(input_ids, max_length=100, num_beams=4, num_beam_groups=2, diversity_penalty=0.5, early_stopping=True, num_return_sequences=1, 
                              eos_token_id=tokenizer.eos_token_id, do_sample=False, top_k=50, top_p=0.95)
    eng_translation = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return eng_translation

## Sentence-Wise

In [63]:
sentence='Wiederaufnahme der Sitzungsperiode'
translate_to_english(sentence)

'Resumption of the session'

## File-Wise

In [66]:
FILE_PATH="sample_infer_de_input.csv"
german_sentences=pd.read_csv("sample_infer_de_input.csv")['de'].tolist()
eng_sentences=[]
for i in tqdm(german_sentences):
    eng_sentences.append(translate_to_english(i))
df = pd.DataFrame({'en': eng_sentences,'de': german_sentences})
df.to_csv('output_partc.csv', index=False)

  0%|          | 0/100 [00:00<?, ?it/s]

C:\Users\Deeptanshu Barman\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\generation\configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
100%|██████████| 100/100 [04:15<00:00,  2.56s/it]
